
# Significant Traces — Batch Pipeline (reads existing dFoF)

Scans experiments (e.g., `L433_f02_Exp_1_flickering`) under your base path, looks in each
`03_analysis/functional/suite2P/plane*/dFoF/` folder for inputs, computes significant traces using
`src/significant_traces.py`, saves arrays & a plot into the same folder, and updates `metadata.json`.


In [5]:

# ==== Configuration ============================================================
from pathlib import Path

BASE_DIR = Path(r"C:\Users\suribear\OneDrive - Université de Lausanne\Lab\Data\2p")
EXPERIMENT_WHITELIST = ['L433_f02_Exp_1_flickering']

FPS_DEFAULT = 2.0
TAUDECAY_DEFAULT = 6.0

ST_N_BINS       = 1000
ST_K_NEIGHBORS  = 100
ST_CONF_CUTOFF  = 90
ST_PLOT_ODDS    = False
ST_VMAX_DFF     = 0.40

SRC_DIR = Path.cwd() / "src"
SHOW_PLOTS_INLINE = True
WRITE_SESSION_SUMMARY = True


In [7]:

# ==== Imports & Helpers ========================================================
import sys, json
from datetime import datetime
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

from src.significant_traces import compute_noise_model_romano_fast_modular, plot_dff_and_raster

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def find_experiments(base_dir: Path, whitelist=None):
    out = []
    for child in base_dir.iterdir():
        if not child.is_dir():
            continue
        if whitelist and child.name not in whitelist:
            continue
        s2p = child / "03_analysis" / "functional" / "suite2P"
        if s2p.exists() and any(p.is_dir() and p.name.startswith("plane") for p in s2p.iterdir()):
            out.append(child)
    return sorted(out, key=lambda p: p.name)

def plane_dirs(exp_dir: Path):
    s2p = exp_dir / "03_analysis" / "functional" / "suite2P"
    return sorted([p for p in s2p.iterdir() if p.is_dir() and p.name.startswith("plane")], key=lambda p: p.name)

def experiment_prefix(exp_name: str) -> str:
    parts = exp_name.split("_")
    if len(parts) >= 2:
        return parts[0] + "_" + parts[1]
    return exp_name

def read_params_from_metadata(dfof_dir: Path, fps_default: float, tauDecay_default: float):
    meta_path = dfof_dir / "metadata.json"
    fps = fps_default
    tauDecay = tauDecay_default
    if meta_path.exists():
        try:
            with open(meta_path, "r", encoding="utf-8") as f:
                meta = json.load(f)
            fps = float(meta.get("fps", fps))
            tauDecay = float(meta.get("tauDecay", meta.get("tau", tauDecay)))
        except Exception:
            pass
    return fps, tauDecay

def update_npz(npz_path: Path, additions: dict):
    payload = {}
    if npz_path.exists():
        try:
            with np.load(npz_path, allow_pickle=False) as z:
                for k in z.files:
                    payload[k] = z[k]
        except Exception:
            payload = {}
    payload.update(additions)
    np.savez_compressed(npz_path, **payload)

def update_metadata(meta_path: Path, st_params: dict, filenames: dict, shapes: dict):
    try:
        meta = {}
        if meta_path.exists():
            with open(meta_path, "r", encoding="utf-8") as f:
                meta = json.load(f)
        st_block = meta.get("significant_traces", {})
        st_block.update({
            "params": st_params,
            "filenames": filenames,
            "shapes": shapes,
            "updated_at": datetime.now().isoformat(timespec="seconds"),
        })
        meta["significant_traces"] = st_block
        with open(meta_path, "w", encoding="utf-8") as f:
            json.dump(meta, f, indent=2)
    except Exception as e:
        print("WARNING: could not update metadata:", e)


In [ ]:

# ==== Batch Run ================================================================
import pandas as pd

session_ts = datetime.now().strftime("%Y%m%d_%H%M%S")
session_root = BASE_DIR / f"session_significant_{session_ts}"
if WRITE_SESSION_SUMMARY:
    ensure_dir(session_root)

experiments = find_experiments(BASE_DIR, whitelist=EXPERIMENT_WHITELIST)
print(f"Experiments found: {len(experiments)}")

summary_rows = []

for exp_dir in experiments:
    exp_name   = exp_dir.name
    prefix     = experiment_prefix(exp_name)
    planes     = plane_dirs(exp_dir)
    print(f"- {exp_name}: {len(planes)} plane(s)")

    for pdir in planes:
        dfof_dir = pdir / "dFoF"
        if not dfof_dir.exists():
            continue

        fps, tauDecay = read_params_from_metadata(dfof_dir, FPS_DEFAULT, TAUDECAY_DEFAULT)

        dFoF_path_candidates = [
            dfof_dir / f"{prefix}_dFoF.npy",
            dfof_dir / f"dFoF_{prefix}.npy"
        ]
        dFoF_path = next((p for p in dFoF_path_candidates if p.exists()), None)
        if dFoF_path is None:
            continue

        dFoF = np.load(dFoF_path)

        (mapOfOdds,
         deltaF_center,
         density_data,
         density_noise,
         xev, yev,
         raster,
         mapOfOddsJoint) = compute_noise_model_romano_fast_modular(
             dFoF,
             n_bins=ST_N_BINS,
             k_neighbors=ST_K_NEIGHBORS,
             confCutOff=ST_CONF_CUTOFF,
             plot_odds=ST_PLOT_ODDS,
             fps=fps,
             tauDecay=tauDecay,
        )

        raster_out        = dfof_dir / f"{prefix}_significant_traces.npy"
        deltaF_center_out = dfof_dir / f"{prefix}_dFoF_center.npy"
        np.save(raster_out, raster)
        np.save(deltaF_center_out, deltaF_center)

        npz_path = dfof_dir / f"{prefix}_dFoF_outputs.npz"
        update_npz(npz_path, {"deltaF_center": deltaF_center, "raster": raster})

        sort_idx = plot_dff_and_raster(deltaF_center, raster, fps=fps, vmax_dff=ST_VMAX_DFF)
        fig = plt.gcf()
        plot_out = dfof_dir / f"{prefix}_significant_traces_plot.png"
        fig.savefig(plot_out, dpi=150, bbox_inches="tight")
        if SHOW_PLOTS_INLINE and pdir == planes[-1]:
            plt.show()
        else:
            plt.close(fig)

        meta_path = dfof_dir / "metadata.json"
        st_params = dict(n_bins=ST_N_BINS, k_neighbors=ST_K_NEIGHBORS, confCutOff=ST_CONF_CUTOFF, fps=fps, tauDecay=tauDecay)
        filenames = dict(raster_npy=raster_out.name, dFoF_center_npy=deltaF_center_out.name, plot_png=plot_out.name, npz_bundle=npz_path.name)
        shapes = dict(deltaF_center=list(deltaF_center.shape), raster=list(raster.shape))
        update_metadata(meta_path, st_params, filenames, shapes)

        summary_rows.append({"experiment": exp_name, "plane": pdir.name, "fps": fps, "tauDecay": tauDecay,
                             "raster_npy": raster_out.name, "dFoF_center_npy": deltaF_center_out.name, "plot_png": plot_out.name})

if WRITE_SESSION_SUMMARY and summary_rows:
    df = pd.DataFrame(summary_rows).sort_values(["experiment", "plane"])
    df_path = session_root / "summary.csv"
    df.to_csv(df_path, index=False, encoding="utf-8")
    print(f"Session summary saved to: {df_path}")
else:
    print("No outputs written (check inputs or whitelist).")


In [9]:

# (Optional) Preview latest session summary
from pathlib import Path
import pandas as pd
try:
    from IPython.display import display
except Exception:
    display = None

sessions = sorted([p for p in BASE_DIR.iterdir() if p.is_dir() and p.name.startswith("session_significant_")])
if sessions:
    latest = sessions[-1]
    csv_path = latest / "summary.csv"
    if csv_path.exists():
        df = pd.read_csv(csv_path)
        if display:
            display(df)
        else:
            print(df.to_string(index=False))
        print(f"Displayed summary from: {csv_path}")
    else:
        print(f"No summary.csv in {latest}")
else:
    print("No session_significant_* folder found yet.")


No session_significant_* folder found yet.
